In [6]:
import numpy as np
import cv2
from mss import mss
from PIL import Image

from copy import copy
import random

import win32gui, win32ui, win32api, win32con

import pygame
import pyautogui
import keyboard

import torch
from torchvision import transforms as T
import torch.nn as nn

import time


pygame 2.1.3 (SDL 2.0.22, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [44]:
class Tile:
    def __init__(self, screen, x, y, w, h, PARENT, value):
        self.screen = screen
        self.rect = pygame.rect.Rect(x, y, w, h)
        self.colour = np.random.randint(0, 30, 3).tolist()
        self._drag = False

        self.rel_x = x-PARENT.x
        self.rel_y = y-PARENT.y

        text_size = min(w,h) // 3

        self._font = pygame.font.SysFont('liberationmono', text_size)

        self.value = value

        self.pressed = False

    def draw(self):
        if self.pressed: colour = (200, 0, 0)
        else: colour = self.colour

        pygame.draw.rect(self.screen, colour, self.rect)

        text_str = f"{self.value:.2f}"

        text = self._font.render(text_str, True, (255, 255, 255))
        text_w = text.get_rect().w
        text_h = text.get_rect().h

        self.screen.blit(text, (self.rect.centerx - text_w//2, self.rect.centery - text_h//2))

class InteractiveFeatureMap:
    def __init__(self, screen, w = 100, h = 100, origin_x = 150, origin_y = 150, tiles_w = 4, tiles_h = 4, colour_bg = (0, 255, 0)):

        self.tiles_w, self.tiles_h = tiles_w, tiles_h
        self.ALL = pygame.rect.Rect(origin_x, origin_y, w, h)

        self.colour_bg = colour_bg
        self.screen = screen

        self.TILES = self.define_tiles()

        self._draging = False

        self._drag_offset_x = None
        self._drag_offset_y = None

        self._initial_mouse_x = None
        self._initial_mouse_y = None

        self._initial_w = None
        self._initial_h = None

        self._initial_pos_x = None
        self._initial_pos_y = None

        self.active_tile = None
        self.active_tile_init_value = None

        self.mouse_last_x, self.mouse_last_y = None, None

        self.max_x, self.max_y = self.screen.get_size()

        self.infinite_screen_counter = 0

    def define_tiles(self, verbose = False):
        tiles = []

        remaining_h = copy(self.ALL.h)
        remaining_tiles_h = copy(self.tiles_h)
        current_y_pos = copy(self.ALL.y)

        for i in range(self.tiles_h):
            remaining_w = copy(self.ALL.w)
            remaining_tiles_w = copy(self.tiles_w)
            current_x_pos = copy(self.ALL.x)

            this_raw = []
            this_h = remaining_h // remaining_tiles_h
            remaining_h -= this_h
            remaining_tiles_h -= 1

            for j in range(self.tiles_w):
                this_w = remaining_w // remaining_tiles_w
                remaining_w -= this_w
                remaining_tiles_w -= 1

                value = random.uniform(-1, 1)

                this_raw.append(Tile(self.screen, current_x_pos, current_y_pos, this_w, this_h, self.ALL, value))

                current_x_pos += this_w
            
            tiles.append(this_raw)
            current_y_pos += this_h

        if verbose:
            for r in tiles:
                print(r)

        return tiles

    def move(self, x, y):

        self.ALL.x = x
        self.ALL.y = y

        for row in self.TILES:
            for tile in row:
                tile.rect.x = x + tile.rel_x; tile.rect.y = y + tile.rel_y

    def draw(self):
        for row in self.TILES:
            for tile in row:
                tile.draw()

    def which_tile_is_triggered(self, position):
        for row in self.TILES:
            for tile in row:
                if tile.rect.collidepoint(position): return tile


    def event_handler(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            self._initial_mouse_x, self._initial_mouse_y = event.pos
            if event.button == 1 and self.ALL.collidepoint(event.pos):   
                tile = self.which_tile_is_triggered(event.pos)
                tile.pressed = True
                self.active_tile_init_value = tile.value
                self.active_tile = tile

                self.mouse_last_x, self.mouse_last_y = event.pos

                self.infinite_screen_counter = 0


        elif event.type == pygame.MOUSEBUTTONUP:
            if event.button == 1:  
                if self.active_tile is not None: 
                    self.active_tile.pressed = False
                self.active_tile_init_value = None
                self.active_tile = None

                self.mouse_last_x, self.mouse_last_y = None, None


        elif event.type == pygame.MOUSEMOTION:
            mouse_x, mouse_y = event.pos

            if self.ALL.collidepoint(event.pos):
                tile = self.which_tile_is_triggered(event.pos)
                


            if self.active_tile is not None: 
                diff_y = mouse_y - self._initial_mouse_y + self.infinite_screen_counter
                
                # infinite dragging upwards
                if mouse_y == 0 and self.mouse_last_y < 20:
                    pyautogui.moveTo(mouse_x, self.max_y)
                    self._initial_mouse_y = self.max_y
                    self.infinite_screen_counter = diff_y

                # infinite dragging upwards
                if mouse_y == (self.max_y-1) and self.mouse_last_y > (self.max_y - 20):
                    pyautogui.moveTo(mouse_x, 0)
                    self._initial_mouse_y = 0
                    self.infinite_screen_counter = diff_y

                self.active_tile.value = self.active_tile_init_value - (diff_y/600)**3

            self.mouse_last_x, self.mouse_last_y = event.pos


In [45]:
class RectBorders:
    def __init__(self, screen, w = 100, h = 100, x = 150, y = 150, b = 7, tiles_w=4, tiles_h=4, colour_bg = (0, 255, 0), colour_border = (255, 0, 0), colour_info = (224, 0, 0)):

        self.b = b #odd number
        self.b2 = int(b/2)

        self.colour_bg = colour_bg
        self.colour_border = colour_border
        self.colour_info = colour_info

        self.screen = screen

        self.MAIN = InteractiveFeatureMap(screen, w = w, h = h, origin_x = x, origin_y = y, tiles_w = tiles_w, tiles_h = tiles_h)

        self.LT = pygame.rect.Rect(self.MAIN.ALL.left-self.b, self.MAIN.ALL.top-self.b, self.b, self.b)
        self.RT = pygame.rect.Rect(self.MAIN.ALL.right,       self.MAIN.ALL.top-self.b, self.b, self.b)
        self.RB = pygame.rect.Rect(self.MAIN.ALL.right,       self.MAIN.ALL.bottom,     self.b, self.b)
        self.LB = pygame.rect.Rect(self.MAIN.ALL.left-self.b, self.MAIN.ALL.bottom,     self.b, self.b)

        self.T  = pygame.rect.Rect(self.MAIN.ALL.left,        self.MAIN.ALL.top-self.b, self.MAIN.ALL.w,  self.b)
        self.R  = pygame.rect.Rect(self.MAIN.ALL.right,       self.MAIN.ALL.top,        self.b,     self.MAIN.ALL.h)
        self.B  = pygame.rect.Rect(self.MAIN.ALL.left,        self.MAIN.ALL.bottom,     self.MAIN.ALL.w,  self.b)
        self.L  = pygame.rect.Rect(self.MAIN.ALL.left-self.b, self.MAIN.ALL.top,        self.b,     self.MAIN.ALL.h)

        self._info_rect2 = pygame.rect.Rect(self.MAIN.ALL.left-self.b, self.MAIN.ALL.bottom+self.b, self.MAIN.ALL.w + self.b + self.b, 20)

        self.ALL = pygame.rect.Rect(x-self.b, y-self.b, (2*self.b)+w, (2*self.b)+h+self._info_rect2.h)

        self._draging = False

        self._drag_offset_x = None
        self._drag_offset_y = None

        self._initial_mouse_x = None
        self._initial_mouse_y = None

        self._initial_w = None
        self._initial_h = None

        self._initial_pos_x = None
        self._initial_pos_y = None

        self._font = pygame.font.SysFont('liberationmono', 18)

        self._last_resize = time.time()

    def move(self, x, y):

        self.MAIN.move(x, y)

        self.LT.x = self.MAIN.ALL.left-self.b;    self.LT.y = self.MAIN.ALL.top-self.b
        self.RT.x = self.MAIN.ALL.right;          self.RT.y = self.MAIN.ALL.top-self.b
        self.RB.x = self.MAIN.ALL.right;          self.RB.y = self.MAIN.ALL.bottom
        self.LB.x = self.MAIN.ALL.left-self.b;    self.LB.y = self.MAIN.ALL.bottom

        self.T.x = self.MAIN.ALL.left;            self.T.y = self.MAIN.ALL.top-self.b;  self.T.w = self.MAIN.ALL.w
        self.R.x = self.MAIN.ALL.right;           self.R.y = self.MAIN.ALL.top;         self.R.h = self.MAIN.ALL.h
        self.B.x = self.MAIN.ALL.left;            self.B.y = self.MAIN.ALL.bottom;      self.B.w = self.MAIN.ALL.w
        self.L.x = self.MAIN.ALL.left-self.b;     self.L.y = self.MAIN.ALL.top;         self.L.h = self.MAIN.ALL.h

        self.ALL.x = x-self.b; self.ALL.y = y-self.b; self.ALL.w = self.MAIN.ALL.w + self.b + self.b; self.ALL.h = self.MAIN.ALL.h + self.b + self.b + self._info_rect2.h

        self._info_rect2.x = self.MAIN.ALL.left-self.b; self._info_rect2.y = self.MAIN.ALL.bottom+self.b; self._info_rect2.w = self.MAIN.ALL.w + self.b + self.b

    def draw(self, info_text):

        self.MAIN.draw()

        pygame.draw.rect(self.screen, self.colour_border, self.LT)
        pygame.draw.rect(self.screen, self.colour_border, self.RT)
        pygame.draw.rect(self.screen, self.colour_border, self.RB)
        pygame.draw.rect(self.screen, self.colour_border, self.LB)

        pygame.draw.rect(self.screen, self.colour_border, self.T)
        pygame.draw.rect(self.screen, self.colour_border, self.R)
        pygame.draw.rect(self.screen, self.colour_border, self.B)
        pygame.draw.rect(self.screen, self.colour_border, self.L)

        pygame.draw.rect(self.screen, self.colour_info, self._info_rect2)

        #### INFO at bottom bar 
        text = self._font.render(info_text, True, (255, 255, 255))
        text_w = text.get_rect().w
        self.screen.blit(text, (self.MAIN.ALL.right-text_w, self.MAIN.ALL.y+self.MAIN.ALL.h+self.b+1))

    def event_handler(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:   
                if self.ALL.collidepoint(event.pos):
                    self._initial_mouse_x, self._initial_mouse_y = event.pos

                    self._drag_offset_x = self.MAIN.ALL.x - self._initial_mouse_x
                    self._drag_offset_y = self.MAIN.ALL.y - self._initial_mouse_y

                    self._initial_w, self._initial_h, self._initial_pos_x, self._initial_pos_y = self.MAIN.ALL.w, self.MAIN.ALL.h, self.MAIN.ALL.x, self.MAIN.ALL.y

                #### MOVE AROUND ####
                if self._info_rect2.collidepoint(event.pos):
                    self._draging = True


        elif event.type == pygame.MOUSEBUTTONUP:
            #reset all resize and move actions
            if event.button == 1:            
                self._draging = False

        elif event.type == pygame.MOUSEMOTION:
            mouse_x, mouse_y = event.pos

            if self._info_rect2.collidepoint(event.pos):
                pygame.mouse.set_cursor(pygame.SYSTEM_CURSOR_SIZEALL)
            elif self.MAIN.ALL.collidepoint(event.pos):
                pygame.mouse.set_cursor(pygame.SYSTEM_CURSOR_SIZENS)
            else:
                pygame.mouse.set_cursor(pygame.SYSTEM_CURSOR_ARROW)

            if self._draging: 
                self.move(mouse_x + self._drag_offset_x, mouse_y + self._drag_offset_y)



In [46]:
class PygameWindow:

    def __init__(self, w, h, tiles_w, tiles_h):

        self.done = False
        self._def_colour = (0, 250, 0)  # Transparency color

        pygame.init()
        self._screen = pygame.display.set_mode((win32api.GetSystemMetrics(0), win32api.GetSystemMetrics(1)), pygame.FULLSCREEN, pygame.NOFRAME) # For borderless, use pygame.NOFRAME
        self._hwnd = pygame.display.get_wm_info()["window"]
        win32gui.SetWindowPos(self._hwnd, win32con.HWND_TOPMOST, 0,0,0,0, win32con.SWP_SHOWWINDOW | win32con.SWP_NOMOVE | win32con.SWP_NOSIZE)
        win32gui.SetWindowLong(self._hwnd, win32con.GWL_EXSTYLE,
                            win32gui.GetWindowLong(self._hwnd, win32con.GWL_EXSTYLE) | win32con.WS_EX_LAYERED)
        win32gui.SetLayeredWindowAttributes(self._hwnd, win32api.RGB(*self._def_colour), 0, win32con.LWA_COLORKEY)
        
        self.rect = RectBorders(self._screen, colour_bg=(0,0,0), w=w, h=h, tiles_w=tiles_w, tiles_h=tiles_h)

    def run(self, info_text):

        self._screen.fill(self._def_colour)  # Transparent background
        keys= pygame.key.get_pressed()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.done = True

            elif event.type == pygame.MOUSEBUTTONDOWN or event.type == pygame.MOUSEBUTTONUP or event.type == pygame.MOUSEMOTION:
                self.rect.event_handler(event)
                self.rect.MAIN.event_handler(event)

        if keys[pygame.K_ESCAPE] or keyboard.is_pressed('esc'):
            self.done = True

        self.rect.draw(info_text)
        pygame.display.update()    

        if self.done: pygame.display.quit()

In [47]:
w = 500
h = 500

pygame_window = PygameWindow(w, h, 4, 4)
while not pygame_window.done:
    pygame_window.run("VQ-VAE sampler")
pygame.display.quit()